In [1]:
cd Desktop/Summer\ research/fem/

/Users/wuyifan/Desktop/Summer research/fem


In [2]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf
import scipy.sparse as sp
import random
import os

name = "cuboid"
path = "data/"+name+"_l"+".obj"
out = "out/out_"+name+"_260"+".mesh"

In [3]:
def add_axis(plot):
    plot.add_lines(np.array([[0,0,0],[50,0,0],[50,0,0]]), np.array([[50,0,0],[48,2,0],[48,-2,0]]), shading={"line_color": "blue", "line_width": 2.0})
    plot.add_lines(np.array([[0,0,0],[0,50,0],[0,50,0]]), np.array([[0,50,0],[-2,48,0],[2,48,0]]), shading={"line_color": "red", "line_width": 2.0})
    plot.add_lines(np.array([[0,0,0],[0,0,50],[0,0,50]]), np.array([[0,0,50],[0,2,48],[0,-2,48]]), shading={"line_color": "yellow", "line_width": 2.0})
    
def add_force(plot, mag, force_ax, ex):
    mag = np.array(mag)
    dir1 = -mag
    dir2 = -mag
    arrow_ax = random.randint(0,2)
    
    while arrow_ax == force_ax:
        arrow_ax = random.randint(0,2)
    rand = np.array([0,0,0]) # random.randint(0,1),random.randint(0,1),random.randint(0,1)
    rand[arrow_ax] = 1
    
    dir1 += rand
    dir2 += -rand
    ex = ps[s[:,0] == force, :]
    mag1 = mag*5
    e11 = np.vstack((np.vstack((ex[0:5]+mag1, ex[0:5]+mag1)), ex[0:5]+mag1))
    e12 = np.vstack((np.vstack((ex[0:5]-mag1, ex[0:5]+mag1+dir1)), ex[0:5]+mag1+dir2))
    plot.add_lines(e11, e12, shading={"line_color": "black"})
    e13 = np.vstack((np.vstack((ex[50:55]+mag1, ex[50:55]+mag1)), ex[50:55]+mag1))
    e14 = np.vstack((np.vstack((ex[50:55]-mag1, ex[50:55]+mag1+dir1)), ex[50:55]+mag1+dir2))
    plot.add_lines(e13, e14, shading={"line_color": "black"})
    
def add_fix(plot):
    plot.add_lines(np.array([[-40, -10, -16],[-40, -10, 16],[40, -10, -16],[-40, -10, -16]]), np.array([[40, -10, -16],[40, -10, 16],[40, -10, 16],[-40, -10, 16]]), shading={"line_color": "brown", "line_width": 5.0})
    #plot.add_points(e2, shading={"point_color": "white", "point_size": 4})

In [4]:
def save(test_save,save_y,test_y,t_uni,save_adj,test_adj,save_x,test_x):
    """
    generate y(target) file
    [stress, disp_x, disp_y, disp_z] """
    if test_save == 0:
        doc1 = open(save_y,'w')
    else:
        doc1 = open(test_y,'w')
    for item in y:
        for i in item:
            print(i, file=doc1, end = ' ')
        print("", file=doc1)
    doc1.close()

    E = np.transpose([t_uni[:,0], t_uni[:,1]])
    E = np.append(E, np.transpose([t_uni[:,0], t_uni[:,2]]))
    E = np.append(E, np.transpose([t_uni[:,0], t_uni[:,3]]))
    E = np.append(E, np.transpose([t_uni[:,1], t_uni[:,2]]))
    E = np.append(E, np.transpose([t_uni[:,1], t_uni[:,3]]))
    E = np.append(E, np.transpose([t_uni[:,2], t_uni[:,3]]))
    l = len(E)
    E.resize((int(l/2),2))
    E = np.unique(E, axis=0)

    """generate edge(-> adj) file"""
    if test_save == 0:
        doc2 = open(save_adj,'w')
    else:
        doc2 = open(test_adj,'w')
    for item in E:
        for i in item:
            print(i, file=doc2, end = ' ')
        print("", file=doc2)
    doc2.close()

    """
    generate x(input feature) file
    [fix, x, y, z, force_x, force_y, force_z] """
    if test_save == 0:
        doc3 = open(save_x,'w')
    else:
        doc3 = open(test_x,'w')
    for item in x:
        for i in item:
            print(i, file=doc3, end = ' ')
        print("", file=doc3)
    doc3.close()

In [20]:
test_save = 1
epoch = 10
    
for i in range(epoch):
    """
    V, F = igl.read_triangle_mesh(path)
    #wm.tetrahedralize(path, out, mute_log=True, edge_length_r=1/10)
    """

    solver = pf.Solver()
    solver.load_mesh_from_path(out, vismesh_rel_area=1e-3)

    v, f = igl.read_triangle_mesh(out)

    minn = np.min(v, axis=0)
    maxx = np.max(v, axis=0)

    #print(len(v))
    #nv = igl.per_vertex_normals(v, f)

    ps, ts, s = solver.get_boundary_sidesets()

    settings = pf.Settings()
    problem = pf.Problem()
    settings.set_pde(pf.PDEs.LinearElasticity)
    settings.set_material_params("E", 10000)
    settings.set_material_params("nu", 0.35)

    #fix = random.randint(1,6)
    fix = 2
    face_idx = [[0,minn[0],"left"], [1,minn[1], "bottom"], [0, maxx[0], "right"], [1, maxx[1], "top"], [2, minn[2], "back"], [2, maxx[2], "front"]]

    force = random.randint(1,6)
    while force == fix:
        force = random.randint(1,6)
    mag = [random.randint(0,10),random.randint(0,10),random.randint(0,10)]
    while mag == [0,0,0]:
        mag = [random.randint(0,3),random.randint(0,3),random.randint(0,3)]
        
    fix_ax = face_idx[fix-1][0]
    fix_val = face_idx[fix-1][1]

    force_ax = face_idx[force-1][0]
    force_val = face_idx[force-1][1]

    # set the displacement value for the sideset id
    problem.set_displacement(fix, [0, 0, 0])
    problem.set_force(force, mag)
    settings.set_problem(problem)
    solver.settings(settings)
    solver.solve()

    p, tri, disp = solver.get_sampled_solution()

    m = np.linalg.norm(disp, axis=1)

    p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)
    t_uni = np.array([inverse[tri[:, 0]], inverse[tri[:, 1]], inverse[tri[:, 2]], inverse[tri[:, 3]]]).transpose()
    d_uni = disp[indices, :]
    m_uni = m[indices]

    mises, stress_ten = solver.get_sampled_mises_avg()
    mises_u = mises[indices, :]

    print(force," ",mag)
    
    """
    plot = mp.plot(p_uni+d_uni, t_uni, mises_u, return_plot=True)
    # plot x,y,z-axis
    add_axis(plot)
    # plot fixed face
    add_fix(plot)
    # plot direction and magnitude of the force
    ex = ps[s[:,0] == force, :]
    add_force(plot, mag, force_ax, ex)
    #plot.save("debug_gr.html")
      """
    
    n = 260

    x = np.zeros([n,7])

    #print(face_idx[force-1][2],mag)

    for i in range(n):
        pt = p_uni[i]
        if pt[fix_ax]==fix_val:
            x[i][0] = 1
        x[i][1] = pt[0]
        x[i][2] = pt[1]
        x[i][3] = pt[2]
        if pt[force_ax]==force_val and not x[i][0]==1:
            x[i][4] = mag[0]
            x[i][5] = mag[1]
            x[i][6] = mag[2]

    y = np.hstack((mises_u,d_uni*1000))

    save_y = "train-force+/y/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"
    save_adj = "train-force+/adj/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"
    save_x = "train-force+/x/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"
    test_y = "test-force+/y/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"
    test_adj = "test-force+/adj/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"
    test_x = "test-force+/x/"+name+" "+str(fix)+" "+str(force)+str(mag)+".txt"

    save(test_save,save_y,test_y,t_uni,save_adj,test_adj,save_x,test_x)

[2020-06-16 19:01:19.703] [polyfem] [info] Loading mesh...
[2020-06-16 19:01:19.703] [geogram] [info] Loading file out/out_cuboid_260.mesh...
[2020-06-16 19:01:19.711] [geogram] [info] (FP64) nb_v:260 nb_e:0 nb_f:386 nb_b:0 tri:1 dim:3
[2020-06-16 19:01:19.711] [geogram] [info]  nb_tets:872
[2020-06-16 19:01:19.711] [geogram] [info] Attributes on vertices: point[3]
[2020-06-16 19:01:19.724] [polyfem] [info] mesh bb min [-30, -10, -6], max [30, 10, 6]
[2020-06-16 19:01:19.724] [polyfem] [info]  took 0.0206338s
[2020-06-16 19:01:19.727] [polyfem] [info] simplex_count: 	872
[2020-06-16 19:01:19.728] [polyfem] [info] regular_count: 	0
[2020-06-16 19:01:19.728] [polyfem] [info] regular_boundary_count: 	0
[2020-06-16 19:01:19.728] [polyfem] [info] simple_singular_count: 	0
[2020-06-16 19:01:19.728] [polyfem] [info] multi_singular_count: 	0
[2020-06-16 19:01:19.728] [polyfem] [info] boundary_count: 	0
[2020-06-16 19:01:19.728] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-06-16 19:

'\n    n,_ = s_uni.shape\n\n    x = np.zeros([n,7])\n    fix_ax = face_idx[fix-1][0]\n    fix_val = face_idx[fix-1][1]\n\n    force_ax = face_idx[force-1][0]\n    force_val = face_idx[force-1][1]\n\n    print(face_idx[force-1][2],mag)\n\n    for i in range(n):\n        pt = p_uni[i]\n        if pt[fix_ax]==fix_val:\n            x[i][0] = 1\n        x[i][1] = pt[0]\n        x[i][2] = pt[1]\n        x[i][3] = pt[2]\n        if pt[force_ax]==force_val and not x[i][0]==1:\n            x[i][4] = mag[0]\n            x[i][5] = mag[1]\n            x[i][6] = mag[2]\n\n    y = np.hstack((mises_u,d_uni*1000))\n\n    save_y = "train/y/"+name+"_"+str(fix)+"_"+str(force)+str(mag)+".txt"\n    save_adj = "train/adj/"+name+"_"+str(fix)+"_"+str(force)+str(mag)+".txt"\n    save_x = "train/x/"+name+"_"+str(fix)+"_"+str(force)+str(mag)+".txt"\n    test_y = "test/y/"+name+"_"+str(fix)+"_"+str(force)+str(mag)+".txt"\n    test_adj = "test/adj/"+name+"_"+str(fix)+"_"+str(force)+str(mag)+".txt"\n    test_x = "t